In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from IPython.display import Markdown
import dotenv
import os

dotenv.load_dotenv()

True

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

embeddings = OpenAIEmbeddings()

In [6]:
dir = "./files/"
file_list = os.listdir(dir)

In [7]:
print(file_list)

['2025-AI-Trend-Report.pdf', 'Practice_document.pdf']


### Part1-1. 문서 기반 챗봇

In [8]:
# 텍스트 스플리터 생성

splitter = RecursiveCharacterTextSplitter(chunk_size=300, 
                                               chunk_overlap=50, 
                                               separators=["\n\n"])

In [9]:
# PDF 로더 생성

loader = PyPDFLoader(dir+file_list[0])

In [10]:
# PDF 문서 로드 및 스플릿

split_docs = loader.load_and_split(text_splitter=splitter)

In [11]:
# 벡터스토어 생성

vector_store = FAISS.from_documents(embedding=embeddings, documents=split_docs)

In [12]:
# 리트리버 생성

retriever = vector_store.as_retriever()

In [13]:
prompt = ChatPromptTemplate([
    ("system", "문서 : {context}\n\n"
               "당신은 문서를 기반으로 응답하는 상담원입니다. 항상 문서를 기반하여 응답해주세요\n"
               "문서에서 응답을 찾을 수 없는 경우 '해당 문의는 010-2222-3333으로 연락해주시면 도와드리겠습니다. 도와드리지 못해 죄송합니다.' 라고 답변하세요."), 
    ("user", "{query}")

])

In [14]:
chain = {"context": retriever | RunnableLambda(lambda docs:"\n\n".join(doc.page_content for doc in docs)),
         "query": RunnablePassthrough()} | prompt | llm

In [15]:
query = "2025년엔 어떤 부분을 준비해야할까?"

In [16]:
result = chain.invoke(query)

In [17]:
print(result.content)

2025년에는 AI 기술의 발전과 함께 기업들이 책임감 있는 AI 활용을 위해 여러 가지를 준비해야 할 것으로 보입니다. 특히 다음과 같은 부분을 준비하는 것이 중요합니다:

1. **신뢰할 수 있는 AI 구축**: 윤리적이고 안전하며 투명한 AI 개발 프로세스를 마련해야 합니다. 이는 기업의 장기적인 성공을 위해 필수적입니다.

2. **AI 기술의 정교화**: 멀티모달 AI와 자율 AI 등 새로운 기술과 서비스를 도입하여 기업의 가치 창출 방식을 변화시켜야 합니다.

3. **AI의 지속 가능성 및 사이버 보안**: AI 기술 활용에서의 지속 가능성과 사이버 보안 문제를 효과적으로 관리하는 노력이 필요합니다.

4. **에너지 효율적인 기술 투자**: AI 시스템의 에너지 소비가 환경에 미치는 영향을 완화하기 위해 에너지 효율적인 칩, 혁신적인 냉각 기술 등에 투자해야 합니다.

5. **AI 신뢰성 검증**: AI의 신뢰성을 효율적으로 검증할 수 있는 시스템을 마련하여 실질적인 인사이트를 얻을 수 있도록 해야 합니다.

이러한 준비를 통해 기업들은 변화하는 환경에 적응하고 경쟁력을 높일 수 있을 것입니다.


In [18]:
result.content

'2025년에는 AI 기술의 발전과 함께 기업들이 책임감 있는 AI 활용을 위해 여러 가지를 준비해야 할 것으로 보입니다. 특히 다음과 같은 부분을 준비하는 것이 중요합니다:\n\n1. **신뢰할 수 있는 AI 구축**: 윤리적이고 안전하며 투명한 AI 개발 프로세스를 마련해야 합니다. 이는 기업의 장기적인 성공을 위해 필수적입니다.\n\n2. **AI 기술의 정교화**: 멀티모달 AI와 자율 AI 등 새로운 기술과 서비스를 도입하여 기업의 가치 창출 방식을 변화시켜야 합니다.\n\n3. **AI의 지속 가능성 및 사이버 보안**: AI 기술 활용에서의 지속 가능성과 사이버 보안 문제를 효과적으로 관리하는 노력이 필요합니다.\n\n4. **에너지 효율적인 기술 투자**: AI 시스템의 에너지 소비가 환경에 미치는 영향을 완화하기 위해 에너지 효율적인 칩, 혁신적인 냉각 기술 등에 투자해야 합니다.\n\n5. **AI 신뢰성 검증**: AI의 신뢰성을 효율적으로 검증할 수 있는 시스템을 마련하여 실질적인 인사이트를 얻을 수 있도록 해야 합니다.\n\n이러한 준비를 통해 기업들은 변화하는 환경에 적응하고 경쟁력을 높일 수 있을 것입니다.'

### Part1-2 고객 응대 챗봇

In [19]:
# PDF 로더 생성

loader = PyPDFLoader(dir+file_list[1])

In [20]:
# PDF 문서 로드 및 스플릿

split_docs = loader.load_and_split(text_splitter=splitter)

In [21]:
# 벡터스토어 생성

vector_store = FAISS.from_documents(embedding=embeddings, documents=split_docs)

In [22]:
# 리트리버 생성

retriever = vector_store.as_retriever()

In [23]:
# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
    context : {context}

    당신은 언제나 고객에게 최선을 다해 답변을 하며 말투는 굉장히 친근합니다. 직업은 전문 상담원입니다. 답변 시, 아래의 규칙을 지켜야만 합니다.
    ---
    ### 규칙 ###
    1. 주어진 context만을 이용하여 답변해야합니다. 
    2. 주어진 context에서 답변을 할 수 없다면 "해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다." 라고 대답하세요.
    3. 문자열에 A1, A2, A11, A22 등 필요 없는 문자는 제거한 뒤 출력합니다.
    4. 항상 친절한 말투로 응대합니다.
    5. 하이퍼 링크를 그대로 출력합니다. 대소문자를 명확하게 구분하세요. 아래 예시를 참고하여 서식을 맞추세요.
    **하이퍼퍼 링크 예시**
    5-1. [스타벅스 구역삼사거리점](https://naver.me/FV7K6xTM) 입니다.
    5-2. [화목순대국](https://naver.me/FQVGK6TZ) 입니다.
    5-3. [모두의연구소 역삼캠퍼스](https://naver.me/GMvc9Hv5) 입니다.
    ---
    """),
    ("human", "{query}")
])

In [24]:
chain = {"context": retriever | RunnableLambda(lambda docs:"\n\n".join(doc.page_content for doc in docs)),
         "query": RunnablePassthrough()} | prompt | llm

In [25]:
query = "개강하는 날짜 알려주세요."

In [26]:
result = chain.invoke(query)

In [27]:
print(result.content)

아이펠 데이터사이언티스트 3기 과정의 개강일은 2024년 9월 25일입니다. 더 궁금한 점이 있으시면 언제든지 말씀해 주세요!


In [28]:
query = "오프라인으로도 수강할 수 있나요?"

In [29]:
result = chain.invoke(query)

In [30]:
print(result.content)

아이펠 교육과정은 100% 실시간 온라인 과정으로 진행됩니다. 하지만 희망하시는 경우, 오프라인 학습 공간인 [모두의연구소 강남 캠퍼스](https://naver.me/FslREXGR) 또는 [모두의연구소 역삼 캠퍼스](https://naver.me/GMvc9Hv5)를 무료로 제공해드리고 있습니다. 공간 사용 방법은 입학 후 별도로 안내드릴 예정입니다.


In [31]:
query = "역삼캠퍼스는 어디에 있나요?"

In [32]:
result = chain.invoke(query)

In [33]:
Markdown(result.content)

역삼캠퍼스는 서울 강남구 역삼로 156 태광빌딩 2층, 4층에 위치해 있습니다. 자세한 내용은 [모두의연구소 역삼캠퍼스](https://naver.me/GMvc9Hv5)에서 확인하실 수 있습니다.

In [34]:
query = "역삼캠퍼스 링크는 뭔가요?"

In [35]:
result = chain.invoke(query)

In [36]:
print(result.content)

역삼캠퍼스 링크는 [모두의연구소 역삼캠퍼스](https://naver.me/GMvc9Hv5) 입니다. 궁금한 점이 있으면 언제든지 물어보세요!
